## 推荐系统
- 音乐数据处理
- 基于商品相似性的推荐-- 协同过滤
- 基于SVD矩阵分解的推荐-- 隐语义模型 

In [1]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    import pandas as pd
import numpy as np
import time
import sqlite3

data_home = './'

## 数据读取


In [2]:
triplet_dataset = pd.read_csv(filepath_or_buffer=data_home+'train_triplets.txt',
                             sep='\t', header=None, names= ['user', 'song', 'play_count'])

In [3]:
triplet_dataset.shape

(48373586, 3)

In [4]:
triplet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48373586 entries, 0 to 48373585
Data columns (total 3 columns):
user          object
song          object
play_count    int64
dtypes: int64(1), object(2)
memory usage: 1.1+ GB


**tip here**:对于int64或float64可以采用astype()转换为int32或float32来减小内存

In [5]:
triplet_dataset.head()

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


## 对原始数据处理
- 计算每个用户的播放总量

In [6]:
output_dict = {}
with open(data_home+'train_triplets.txt') as f:
    for line_num, line in enumerate(f):
        user = line.split('\t')[0]
        play_count = int(line.split('\t')[2])
        if user in output_dict:
            play_count += output_dict[user]
            output_dict.update({user:play_count}) # update当前指标加上原指标
        output_dict.update({user:play_count})
output_list = [{'user':k, 'play_count':v} for k, v in output_dict.items()]
play_count_df = pd.DataFrame(output_list)
play_count_df = play_count_df.sort_values(by = 'play_count', ascending=False)

In [7]:
play_count_df.to_csv(path_or_buf='user_playcount_df.csv', index=False)

- 对每首歌计算其播放总量

In [8]:
output_dict = {}
with open(data_home+'train_triplets.txt') as f:
    for line_number, line in enumerate(f):
        song = line.split('\t')[1]
        play_count = int(line.split('\t')[2])
        if song in output_dict:
            play_count += output_dict[song]
            output_dict.update({song:play_count})
        output_dict.update({song:play_count})
output_list = [{'song':k, 'play_count':v} for k, v in output_dict.items()]
song_count_df = pd.DataFrame(output_list)
song_count_df = song_count_df.sort_values(by='play_count', ascending=False)

In [9]:
song_count_df.to_csv(path_or_buf='song_playcount_df.csv', index=False)

In [10]:
play_count_df = pd.read_csv(filepath_or_buffer='user_playcount_df.csv')
play_count_df.head()

,user,play_count
0,093cb74eb3c517c5179ae24caf0ebec51b24d2a2,13132
1,119b7c88d58d0c6eb051365c103da5caf817bea6,9884
2,3fa44653315697f42410a30cb766a4eb102080bb,8210
3,a2679496cd0af9779a92a13ff7c6af5c81ea8c7b,7015
4,d7d2d888ae04d16e994d6964214a1de81392ee04,6494


In [11]:
song_count_df = pd.read_csv('song_playcount_df.csv')
song_count_df.head()

,song,play_count
0,SOBONKR12A58A7A7E0,726885
1,SOAUWYT12A81C206F1,648239
2,SOSXLTC12AF72A7F54,527893
3,SOFRQTD12A81C233C0,425463
4,SOEGIYH12A6D4FC0E3,389880


### 排序的目的：
tip:惰性用户： 播放数较少，值时体验的用户，没有推荐的必要将其过滤掉

    取10W用户，3W歌

In [12]:
total_play_count = sum(song_count_df.play_count)  # 总播放数
total_play_count

138680243

In [13]:
print(float(play_count_df.head(n=100000).play_count.sum()/total_play_count)*100)
# 前100000个用户的播放量占比
play_count_subset = play_count_df.head(100000)

40.8807280500655


In [14]:
# 前30000首歌
song_count_subsest = song_count_df.head(30000)
float(song_count_df.head(30000).play_count.sum()/total_play_count*100)

78.39315366645269

### 过滤其他用户

In [15]:
user_subset = list(play_count_subset.user)
song_subset = list(song_count_subsest.song)

In [16]:
# del(triplet_dataset)

In [17]:
# triplet_dataset = pd.read_csv(filepath_or_buffer=data_home+'train_triplets.txt', sep='\t', header=None, names=['user', 'song', 'play_count'])
# triplet_dataset_ = pd.read_csv(filepath_or_buffer=data_home+'train_triplets.txt', sep='\t', header=None, names= ['user', 'song', 'play_count'])

In [18]:
triplet_subset = triplet_dataset[triplet_dataset.user.isin(user_subset)]
del(triplet_dataset)

In [19]:
triplet_subset_subsong = triplet_subset[triplet_subset.song.isin(song_subset)]
del(triplet_subset)

In [20]:
triplet_subset_subsong.to_csv('triplet_subset_subsong.csv', index=False)

## 我们生成的数据集

In [21]:
triplet_subset_subsong.shape

(10774558, 3)

In [22]:
triplet_subset_subsong.head(10)

,user,song,play_count
498,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12
499,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1
500,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1
501,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1
502,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7
503,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBONKR12A58A7A7E0,26
504,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBZZDU12A6310D8A3,7
505,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOCAHRT12A8C13A1A4,5
506,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODASIJ12A6D4F5D89,1
507,d6589314c0a9bcbca4fee0c93b14bc402363afea,SODEAWL12AB0187032,8


## 数据集整合

    - 补全音乐名字：sqlite3， .db转换为.csv

In [23]:
conn = sqlite3.connect(data_home+'track_metadata.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('songs',)]

In [24]:
# pandas 执行sql操作
track_metadata_df = pd.read_sql(sql='select * from songs', con=conn)
track_metadata_df.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


In [25]:
track_metadata_df_sub = track_metadata_df[track_metadata_df.song_id.isin(song_subset)]
# 在元数据中的所有歌曲名取出

In [26]:
track_metadata_df_sub.to_csv('track_metadata_df_sub.csv')

In [27]:
track_metadata_df_sub.shape  # 有重复的

(30447, 14)

### 整合
pandas的merge操作， song_id(track_metadata_df_sub)和song(triplet_dataset_subsong)

In [28]:
track_metadata_df_sub.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
115,TRMMGCB128E079651D,Get Along (Feat: Pace Won) (Instrumental),SOHNWIM12A67ADF7D9,Charango,ARU3C671187FB3F71B,067102ea-9519-4622-9077-57ca4164cfbb,Morcheeba,227.47383,0.819087,0.533117,2002,185967,-1,0
123,TRMMGTX128F92FB4D9,Viejo,SOECFIW12A8C144546,Caraluna,ARPAAPH1187FB3601B,f69d655c-ffd6-4bee-8c2a-3086b2be2fc6,Bacilos,307.51302,0.595554,0.400705,0,6825058,-1,0
145,TRMMGDP128F933E59A,I Say A Little Prayer,SOGWEOB12AB018A4D0,The Legendary Hi Records Albums_ Volume 3: Ful...,ARNNRN31187B9AE7B7,fb7272ba-f130-4f0a-934d-6eeea4c18c9a,Al Green,133.58975,0.779490,0.599210,1978,5211723,-1,11898
172,TRMMHBF12903CF6E59,At the Ball_ That's All,SOJGCRL12A8C144187,Best of Laurel & Hardy - The Lonesome Pine,AR1FEUF1187B9AF3E3,4a8ae4fd-ad6f-4912-851f-093f12ee3572,Laurel & Hardy,123.71546,0.438709,0.307120,0,8645877,-1,0
191,TRMMHKG12903CDB1B5,Black Gold,SOHNFBA12AB018CD1D,Total Life Forever,ARVXV1J1187FB5BF88,6a65d878-fcd0-42cf-aff9-ca1d636a8bcc,Foals,386.32444,0.842578,0.514523,2010,9007438,-1,0


In [29]:
# 清洗无用数据
track_data = track_metadata_df_sub.loc[:,('song_id', 'title','release', 'artist_name', 'year')]
# del(track_metadata_df_sub.iloc[:, track_metadata_df_sub.columns.isin(['song_id', 'title'])])

In [30]:
track_data.shape

(30447, 5)

In [31]:
# 在元数据中取了3W首歌， 去除重复数据
track_data = track_data.drop_duplicates(['song_id'])
track_data.shape

(30000, 5)

In [32]:
# 合并两张表
triplet_dataset_sub_song_merged = pd.merge(triplet_subset_subsong, track_data, 
                                           how='left', left_on='song', right_on='song_id')

In [33]:
# 改名
triplet_dataset_sub_song_merged = triplet_dataset_sub_song_merged.rename(columns={'play_count':'listen_count'})

MemoryError: Unable to allocate array with shape (6, 10774558) and data type object

In [ ]:
triplet_dataset_sub_song_merged.head()

In [ ]:
triplet_dataset_sub_song_merged.drop(['song_id'], axis=1)

## EDA

In [ ]:
# 最流行
popular_songs = triplet_dataset_sub_song_merged[['title', 'listen_count']].groupby('title').sum().reset_index()

In [ ]:
popular_songs.shape

In [ ]:
popular_songs_top_20 = popular_songs.sort_values(by='listen_count', ascending=False).head(20)
popular_songs_top_20

In [ ]:
objects = (list(popular_songs_top_20['title']))
x_pos = np.arange(len(objects))
performance = list(popular_songs_top_20['listen_count'])

import matplotlib.pyplot as plt


In [ ]:
# 条形图
plt.bar(x_pos, performance, align='center', alpha=0.5)
plt.xticks(x_pos, objects, rotation='vertical')
plt.ylabel('Item count')
plt.title('Most popular songs')

plt.show()

In [ ]:
# 最欢迎的歌手
popular_artists = triplet_dataset_sub_song_merged.loc[:, ('artist_name','listen_count')].groupby(
    'artist_name').sum().reset_index()

In [ ]:
popular_artists.head()

In [ ]:
popular_artists_top_20 = popular_artists.sort_values(by='listen_count', ascending=False).head(20)

In [ ]:
objects = list(popular_artists_top_20['artist_name'])
x_pos = list(np.arange(len(popular_artists_top_20)))
listen = list(popular_artists_top_20['listen_count'])

plt.bar(x_pos, listen, align='center', alpha=0.5)
plt.xticks(x_pos, objects, rotation='vertical')
plt.ylabel('listen_counts')
plt.title('most popular artists')

plt.show()

### 用户分布量的分布


In [ ]:
user_song_count_distribution = triplet_dataset_sub_song_merged[['user', 'title']].groupby(
    'user').count().reset_index().sort_values(by='title', ascending=False)
# 每位用户听歌数量

In [ ]:
user_song_count_distribution.title.describe()

In [ ]:
user_song_count_distribution.head()

In [ ]:
x = user_song_count_distribution.title

n, bins, patched = plt.hist(x, 50)
plt.xlabel('play counts')
plt.ylabel('Nums of users')
plt.title(r'$\mathrm{Histogram\ of\ User \ Play\ Count\ Distribution\ }$')
plt.grid(True)
plt.show()

## **Core**:推荐系统


In [ ]:
import Reconmmenders as Reconmmenders
from sklearn.model_selection import train_test_split

In [ ]:
triplet_dataset_sub_song_merged_set = triple_dataset_sub_song_merged
train_data, test_data = train_test_split(triplet_dataset_sub_song_merged_set, test_size=0.3, random_state=0)

In [ ]:
def create_popularity_recommendation(train_data, user_id, item_id):
    #Get a count of user_ids for each unique song as recommendation score
    train_data_grouped = train_data.groupby([item_id]).agg({usesr_id:'count'}).reset_index()
    train_data_grouped.rename(columns={user_id:'score'}, inplace=True)
    
    #Sort the songs based on Score
    train_data_sort = train_data_grouped.sort_values(by='score', ascending=False)
    
    #Generate a recommendation rank based on score
    train_data_sort['rank'] = train_data_grouped['score'].rank(ascending=False, method='first')
    
    # Get top 20
    popularity_recommendations = train_data_sort.head(20)
    return popularity_recommendations